# 한빛아카데미

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import urllib.request
import pandas as pd

# conda install -c conda-forge tqdm
from tqdm import tqdm
from tqdm import tqdm_notebook
import time

In [2]:
for i in tqdm(range(10)):
    time.sleep(.1)

100%|██████████| 10/10 [00:01<00:00,  9.05it/s]


In [3]:
for i in tqdm_notebook(range(10)):
    time.sleep(.1)

In [4]:
import requests
url = 'https://www.hanbit.co.kr/academy/books/new_book_list.html'
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36'}
req = requests.get(url, headers = header)
soup = BeautifulSoup(req.text, 'html.parser')

In [5]:
import re

data = []

for i, row in enumerate(soup.select('#container .sub_book_list')):
    
    data.append({})

    # P_CODE
    element = row.select_one('.pop_quick_bg')
    attr = element.get('onclick')
    match = re.compile(r"p_code=(\S+\d+)").search(attr)
    text = match.group(1) if match != None else 0
    data[i].update({'p_code':text})

    # TITLE
    element = row.select_one('.book_tit')
    text = element.get_text(strip=True)
    data[i].update({'title':text})

    # WRITER
    element = row.select_one('.book_writer')
    text = element.get_text(strip=True)
    data[i].update({'writer':text})

    # PRICE
    element = row.select_one('.price')
    text = element.get_text(strip=True)
    text = int(text.replace(',', '').replace('원', ''))
    data[i].update({'price':text})

In [6]:
data[:1]

[{'p_code': 'B4797877052',
  'title': '포르잔 C++ 에센셜',
  'writer': 'Behrouz A. Forouzan , Richard F. Gilberg',
  'price': 32000}]

In [7]:
[i for i in data if i.get('p_code') == 'B4797877052']

[{'p_code': 'B4797877052',
  'title': '포르잔 C++ 에센셜',
  'writer': 'Behrouz A. Forouzan , Richard F. Gilberg',
  'price': 32000}]

In [8]:
p_codes = [i['p_code'] for i in data]
v_data = []

for i, p_code in enumerate(p_codes):

    view_base = 'https://www.hanbit.co.kr/academy/books/book_view.html?p_code='
    view_url = view_base + p_code

    with urllib.request.urlopen(view_url) as response:
        html = response.read()
        soup = BeautifulSoup(html, 'html.parser')

        v_data.append({})

        # P_CODE
        v_data[i].update({ 'p_code': p_code })

        # TITLE
        eKey = soup.select_one('#container .store_product_info_box h3')
        oValue = eKey.get_text(strip=True)
        v_data[i].update({ 'title': oValue })

        # INFO_LIST
        for row in soup.select('#container .info_list > li'):

            # key
            eKey = row.select_one('strong')
            oKey = eKey.get_text(strip=True)
            oKey = oKey.replace(' :', '')
            
            # value
            eValue = row.select_one('span')
            oValue = eValue.get_text(strip=True)

            # Exception
            if oKey in '저자':
                oValue = ', '.join(map(str.strip, oValue.split(',')))

            if oKey in '페이지':
                oValue = oValue.replace('쪽', '').strip()

            # output
            v_data[i].update({ oKey: oValue })

In [9]:
df1 = pd.DataFrame(data)
df2 = pd.DataFrame(v_data)
df = pd.merge(df1, df2, on='p_code')

# Drop and Replace
df.drop(['title_y', 'writer'], inplace=True, axis=1)
df.rename(columns={
    'title_x': '제목',
    'price': '가격',
}, inplace=True)

# Reorder
df = df[['p_code', 'ISBN', '제목', '가격', '저자', '번역', '출간', '페이지', '구판정보', '물류코드']]
df

,p_code,ISBN,제목,가격,저자,번역,출간,페이지,구판정보,물류코드
0,B4797877052,9791156645061,포르잔 C++ 에센셜,32000,"Behrouz A. Forouzan, Richard F. Gilberg",윤인성,2020-11-10,756,NaN,4506
1,B6503201303,9791156645054,"IT CookBook, 액세스 2019로 배우는 데이터베이스 기초와 실습",28000,김지연,NaN,2020-09-28,536,"이 도서는 <IT CookBook, 데이터베이스 기초와 실습 : 액세스 2013>의...",4505
2,B1851418066,9791156645030,포르잔 C++ 바이블,45000,"Behrouz A. Forouzan, Richard F. Gilberg",윤인성,2020-08-31,1192,NaN,4503
3,B9479195027,9791156644972,딥러닝을 위한 선형대수학,38000,Gilbert Strang,부산대학교 산업수학센터(FFMIMC),2020-08-05,560,NaN,4497
4,B2082711299,9791156645023,"IT CookBook, 오라클로 배우는 데이터베이스 개론과 실습(2판)",29000,"박우창, 남송휘, 이현룡",NaN,2020-07-30,600,"이 도서는 <IT CookBook, 오라클로 배우는 데이터베이스 개론과 실습>의 개...",4502
5,B1825116777,9791156645016,대학을 버려야 대학이 산다,18000,조동성 외 인천대학교 가족 178명,NaN,2020-07-25,216,NaN,4501
6,B5937482728,9791156644941,"IT CookBook, 따라 하면서 배우는 사물인터넷",28000,양재삼,NaN,2020-07-23,448,NaN,4494
7,B8404138031,9791156644989,R로 하는 다변량 데이터 분석 : 이론부터 실무 활용까지,32000,"카와하시 잇코, 이와마 노리카즈, 스즈키 마사유키",권기태,2020-07-10,544,NaN,4498
8,B8573612142,9791156644934,창의적 공학설계(3판),23000,김은경,NaN,2020-07-10,408,이 도서는 <창의적 공학설계(개정판)>의 개정판입니다.구판 정보 보기,4493
9,B8658821149,9791156644996,공학 핵심수학(2판) : 기초부터 심화까지 핵심만 쏙쏙,23000,함남우,NaN,2020-07-06,484,이 도서는 <공학 핵심수학 : Core Mathematics for Engineer...,4499


## 페이지네이션

In [10]:
import requests
base_url = 'https://www.hanbit.co.kr/academy/books/new_book_list.html'
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36'}
req = requests.get(base_url, headers = header)
soup = BeautifulSoup(req.text, 'html.parser')

In [11]:
import re

def get_next_page(soup):
    page_nums = []
    for i, row in enumerate(soup.select('div.paginate > a')):
        match = re.compile(r"page=(\d+)").search(str(row))
        txt = int(match.group(1)) if match != None else 0
        page_nums.append(txt)
    return max(page_nums)

def is_max_page(page):

    page = page + 1
    web_url = base_url + '?page=' + str(page)

    with urllib.request.urlopen(web_url) as response:
        html = response.read()
        soup = BeautifulSoup(html, 'html.parser')
        
        nolist = soup.select('#container .sub_book_list_area .no-list')
        islist = (False if len(nolist) == 0 else True)

        return islist

def get_max_page(page):

    page = page + 1
    web_url = base_url + '?page=' + str(page)

    with urllib.request.urlopen(web_url) as response:
        html = response.read()
        soup = BeautifulSoup(html, 'html.parser')
        last = get_next_page(soup)

        if is_max_page(last):
            return last
        else:
            return get_max_page(last)

In [12]:
data = {
    'head' : [],
    'body' : []
}

page_max = get_max_page(get_next_page(soup))

for page in range(0, page_max):

    page = page + 1
    web_url = base_url + '?page=' + str(page)

    page_size = len(data['body'])

    with urllib.request.urlopen(web_url) as response:
        html = response.read()
        soup = BeautifulSoup(html, 'html.parser')

        for i, row in enumerate(soup.select('#container .sub_book_list')):
            
            idx = page_size + i
            data['body'].append({})

            # P_CODE
            element = row.select_one('.pop_quick_bg')
            attr = element.get('onclick')
            match = re.compile(r"p_code=(\S+\d+)").search(attr)
            text = match.group(1) if match != None else 0
            data['body'][i].update({'p_code':text})

            # TITLE
            element = row.select_one('.book_tit')
            text = element.get_text(strip=True)
            data['head'].append(txt)
            data['body'][i].update({'title':text})

            # WRITER
            element = row.select_one('.book_writer')
            text = element.get_text(strip=True)
            data['body'][i].update({'writer':text})

            # PRICE
            element = row.select_one('.price')
            text = element.get_text(strip=True)
            text = int(text.replace(',', '').replace('원', ''))
            data['body'][i].update({'price':text})

NameError: name 'txt' is not defined

In [9]:
len(data['body'])

301

In [10]:
df = pd.DataFrame(data['body'])
df.sort_values(by="price", ascending=False)
df

,p_code,title,writer,price
0,B4797877052,포르잔 C++ 에센셜,"Behrouz A. Forouzan , Richard F. Gilberg",32000
1,B6503201303,"IT CookBook, 액세스 2019로 배우는 데이터베이스 기초와 실습",김지연,28000
2,B1851418066,포르잔 C++ 바이블,"Behrouz A. Forouzan , Richard F. Gilberg",45000
3,B9479195027,딥러닝을 위한 선형대수학,Gilbert Strang,38000
4,B2082711299,"IT CookBook, 오라클로 배우는 데이터베이스 개론과 실습(2판)","박우창 , 남송휘 , 이현룡",29000
...,...,...,...,...
296,B5468706684,문제해결력을 키우는 이공계 글쓰기 : 발표와 토론까지,한성우,18000
297,B1672014946,"진로개발과 직업세계 : 대학생과 청년층을 위한 진로, 취업지침서","서일수 , 김도균 , 김지혜 , 이정배",20000
298,B4698900723,Essential Calculus,James Stewart,36000
299,B2947348985,"IT CookBook, 디지털 통신 시스템 : Digital Communicatio...","정호영 , 임승각",25000
